# Unified Shared Memory (USM) (Fortran)

#### Sections
- [Learning Objectives](#Learning-Objectives)
- [Allocating Unified Shared Memory](#Allocating-Unified-Shared-Memory)
- _Code:_ [Lab Exercise: Shared Memory Allocation ](#Lab-Exercise:-Shared-Memory-Allocation)

## Learning Objectives
* Use the Unified Shared Memory feature to simplify OpenMP* Offload programming
* Understand implicit and explicit way of moving memory using USM

### Prerequisites
Basic understanding of OpenMP constructs are assumed for this module. You also should have already went through the  [Introduction to OpenMP Offload module](../intro/intro_f.ipynb) and [Managing Device Data module](../datatransfer/datatransfer_f.ipynb), where the basics of using the Jupyter notebooks with the Intel® DevCloud and an introduction to the OpenMP `target` and `map` constructs were discussed.

## What is Unified Shared Memory?

Unified Shared Memory (USM) is a tool for data management currently supported by the Intel&reg; Fortran Compiler. USM __simplifies development__ for the programmer when __porting existing code__ to support OpenMP Offload.

### Developer View of USM

The picture below shows __developer view of memory__ without USM and with USM. 

With USM, the developer can reference that same memory object in host and device code.  

![Developer View of USM](Assets/usm_dev_view.png)

***
## Allocating Unified Shared Memory
In the previous modules, we used the `map` clause with the `target`, `target data`, and `target enter/exit data` pragmas which enabled the mapping of memory between host and device data environments. However, we can also use OpenMP USM routines to simplify the management of host and device memories.

### Types of USM

USM provides different types of memory to allow both explicit and implicit models for managing memory.

Device memory can be allocated for explicit user control of data movement. Host and shared memory are provided to allow implicit accesses from the accelerator device.

The following table illustrates the properties of the different USM memories and how they can be allocated. The allocate directive needs to be used prior to the allocate command.

|Type | Location | Accessible From |  allocate directive |
|:----:|:----:|:----:|:----|
|Host | Host | Host or Device | !\$omp allocate allocator(omp_target_host_mem_alloc) |
|Device |Device | Device | !\$omp allocate allocator(omp_target_device_mem_alloc) |
|Shared | Host or Device | Host or Device | !\$omp allocate allocator(omp_target_shared_mem_alloc) |

## Lab Exercise: Shared Memory Allocation 

In this exercise, you will use the shared allocation routine to highlight the usage of Unified Shared Memory. Shared memory is accessible from both the host and device. Its location is managed by the runtime and can reside on the host and/or the device.

The primary source file, main.cpp, is written for you. 
It includes alloc_func.cpp that you will write out. If you would like to see the contents of main.cpp, execute the following cell.


In [ ]:
%%writefile lab/main.f90
!==============================================================
! Copyright © 2020 Intel Corporation
!
! SPDX-License-Identifier: MIT
! =============================================================
program main
    use omp_lib
    integer, parameter :: N=16
    integer :: i
    integer, allocatable :: x(:)
    logical :: is_cpu = .true.

    !$omp allocate allocator(omp_target_shared_mem_alloc)
    allocate(x(N)) 
    
    do i=1,N
        x(i) = i
    end do  

    !$omp target map(tofrom: is_cpu) has_device_addr(x)
    !$omp teams distribute parallel do
    do i=1,N
        if ((i==1) .and. (.not.(omp_is_initial_device()))) is_cpu=.false.
        x(i) = x(i) * 2
    end do
    !$omp end target
        
    if (is_cpu) then
        print *, "Running on CPU"
    else
        print *, "Running on GPU"
    end if
        
    do i=1,N
        print *, x(i)
    end do
    
    deallocate(x)
end program main

### Compile and Execute the code
Next, compile and run the code using the script *run.sh*.

In [ ]:
# Optional: Run this cell to see the contents of run.sh
%pycat run.sh

Execute the following cell to execute main.cpp. Look for the passed message.

_If the Jupyter cells are not responsive or if they error out when you compile the samples, please restart the Kernel and compile the samples again_

In [ ]:
! chmod 755 q; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q run.sh; else ./run.sh; fi

# Summary
USM makes it easy to use OpenMP Offload. USM allows a simple implicit data movement approach to get functional quickly. USM also provides controlled data movement with explicit approach.

<html><body><span style="color:Red"><h1>Reset Notebook</h1></span></body></html>

##### Should you be experiencing any issues with your notebook or just want to start fresh run the below cell.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/OpenMP_Offload/datatransfer/ ~/OpenMP_Offload/datatransfer
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

***

@Intel Corporation | [\*Trademark](https://www.intel.com/content/www/us/en/legal/trademarks.html)